In [1]:
from __future__ import print_function
from ucimlrepo import fetch_ucirepo 
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import warnings
import statsmodels.api as sm
import pandas as pd

warnings.filterwarnings("ignore")

In [2]:
icustays = pd.read_csv('data/icustays.csv')
patients = pd.read_csv('data/patients.csv')
transfers = pd.read_csv('data/transfers.csv')
admissions = pd.read_csv('data/admissions.csv')
diagnoses = pd.read_csv('data/diagnoses_icd.csv')
d_items = pd.read_csv('data/d_items.csv')

# 데이터 전처리

> ## patients 전처리

In [3]:
# 분석에 쓸모 없는 열 삭제
patients = patients.drop(['anchor_year', 'anchor_year_group'], axis=1)

# 사망 여부는 예측 시점에서 알 수 없는 정보이기 때문에 삭제
patients = patients.drop(['dod'], axis=1)

# 남성이면 1, 여성이면 0
patients['gender'] = (patients['gender'] == 'M') * 1

In [4]:
# patients 테이블에서 사용할 feature들 출력
patients.head()

,subject_id,gender,anchor_age
0,10000048,0,23
1,10002723,0,0
2,10003939,1,0
3,10004222,1,0
4,10005325,0,0


In [5]:
# patients 테이블에 결측치가 없음을 확인함
patients.isnull().sum()

subject_id    0
gender        0
anchor_age    0
dtype: int64

> ## admissions 전처리

In [6]:
# 분석에 쓸모 없는 열과 예측 시점에서 알 수 없는 열을 삭제
admissions = admissions.drop(['language', 'admittime', 'dischtime', 'deathtime', 'discharge_location', 'marital_status', 'edregtime', 'edouttime', 'hospital_expire_flag'], axis=1)

In [7]:
admissions = pd.get_dummies(admissions, columns=['admission_type'])
admissions = pd.get_dummies(admissions, columns=['admission_location'])
admissions = pd.get_dummies(admissions, columns=['insurance'])
admissions = pd.get_dummies(admissions, columns=['ethnicity'])

In [8]:
admissions.head()

,subject_id,hadm_id,admission_type_AMBULATORY OBSERVATION,admission_type_DIRECT EMER.,admission_type_DIRECT OBSERVATION,admission_type_ELECTIVE,admission_type_EU OBSERVATION,admission_type_EW EMER.,admission_type_OBSERVATION ADMIT,admission_type_SURGICAL SAME DAY ADMISSION,...,insurance_Medicare,insurance_Other,ethnicity_AMERICAN INDIAN/ALASKA NATIVE,ethnicity_ASIAN,ethnicity_BLACK/AFRICAN AMERICAN,ethnicity_HISPANIC/LATINO,ethnicity_OTHER,ethnicity_UNABLE TO OBTAIN,ethnicity_UNKNOWN,ethnicity_WHITE
0,14679932,21038362,False,False,False,True,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
1,15585972,24941086,False,False,False,True,False,False,False,False,...,False,True,False,False,False,False,False,False,False,True
2,11989120,21965160,False,False,False,True,False,False,False,False,...,False,True,False,False,False,False,False,False,True,False
3,17817079,24709883,False,False,False,True,False,False,False,False,...,False,True,False,False,False,False,True,False,False,False
4,15078341,23272159,False,False,False,True,False,False,False,False,...,False,True,False,False,True,False,False,False,False,False


In [9]:
# admission 테이블에 결측치가 없음을 확인함

admissions.isnull().sum()

subject_id                                                   0
hadm_id                                                      0
admission_type_AMBULATORY OBSERVATION                        0
admission_type_DIRECT EMER.                                  0
admission_type_DIRECT OBSERVATION                            0
admission_type_ELECTIVE                                      0
admission_type_EU OBSERVATION                                0
admission_type_EW EMER.                                      0
admission_type_OBSERVATION ADMIT                             0
admission_type_SURGICAL SAME DAY ADMISSION                   0
admission_type_URGENT                                        0
admission_location_AMBULATORY SURGERY TRANSFER               0
admission_location_CLINIC REFERRAL                           0
admission_location_EMERGENCY ROOM                            0
admission_location_INFORMATION NOT AVAILABLE                 0
admission_location_INTERNAL TRANSFER TO OR FROM PSYCH  

> ## transfers 전처리

In [10]:
# 분석에 쓸모 없는 열과 예측 시점에서 알 수 없는 열을 삭제
transfers = transfers.drop(['intime', 'outtime', 'transfer_id'], axis=1)

In [11]:
transfers = pd.get_dummies(transfers, columns=['eventtype'])
# transfers = pd.get_dummies(transfers, columns=['careunit']) # 카디널리티 42
transfers = transfers.drop(['careunit', 'eventtype_discharge'], axis=1)

In [12]:
transfers

,subject_id,hadm_id,eventtype_ED,eventtype_admit,eventtype_transfer
0,14550633,24512724.0,False,True,False
1,15216953,25759433.0,False,True,False
2,15776555,28490372.0,False,True,False
3,12248661,23426497.0,False,False,False
4,14261068,20086032.0,False,False,False
...,...,...,...,...,...
2189530,13152507,NaN,True,False,False
2189531,13575285,NaN,True,False,False
2189532,16414198,20282129.0,False,True,False
2189533,15596627,26100831.0,False,True,False


In [13]:
# transfers 데이터프레임의 각 변수(열)별 결측치 개수 세기

transfers.isnull().sum()

subject_id                 0
hadm_id               348553
eventtype_ED               0
eventtype_admit            0
eventtype_transfer         0
dtype: int64

In [14]:
# 'hadm_id'는 환자의 입원 id이므로 해당 값을 대치법을 통해 채우면 데이터의 왜곡이 발생
# 그래서 그냥'hadm_id'에 결측치가 있는 행은 제거하기로 함

# 'hadm_id' 열에서 결측치가 있는 행 제거
transfers = transfers.dropna(subset=['hadm_id'])

In [15]:
transfers = transfers.dropna()
print(transfers.isna().sum())

subject_id            0
hadm_id               0
eventtype_ED          0
eventtype_admit       0
eventtype_transfer    0
dtype: int64


In [16]:
transfers.head()

,subject_id,hadm_id,eventtype_ED,eventtype_admit,eventtype_transfer
0,14550633,24512724.0,False,True,False
1,15216953,25759433.0,False,True,False
2,15776555,28490372.0,False,True,False
3,12248661,23426497.0,False,False,False
4,14261068,20086032.0,False,False,False


> ## icustays 전처리

In [17]:
icustays

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
0,17867402,24528534,31793211,Trauma SICU (TSICU),Trauma SICU (TSICU),2154-03-03 04:11:00,2154-03-04 18:16:56,1.587454
1,14435996,28960964,31983544,Trauma SICU (TSICU),Trauma SICU (TSICU),2150-06-19 17:57:00,2150-06-22 18:33:54,3.025625
2,17609946,27385897,33183475,Trauma SICU (TSICU),Trauma SICU (TSICU),2138-02-05 18:54:00,2138-02-15 12:42:05,9.741725
3,18966770,23483021,34131444,Trauma SICU (TSICU),Trauma SICU (TSICU),2123-10-25 10:35:00,2123-10-25 18:59:47,0.350544
4,12776735,20817525,34547665,Neuro Stepdown,Neuro Stepdown,2200-07-12 00:33:00,2200-07-13 16:44:40,1.674769
...,...,...,...,...,...,...,...,...
76535,15368898,27299174,39990887,Trauma SICU (TSICU),Trauma SICU (TSICU),2126-06-13 01:00:00,2126-06-13 20:28:35,0.811516
76536,15721773,28911582,39991872,Medical Intensive Care Unit (MICU),Medical Intensive Care Unit (MICU),2177-11-08 14:09:00,2177-11-10 00:24:54,1.427708
76537,12275003,22562812,39992247,Cardiac Vascular Intensive Care Unit (CVICU),Cardiac Vascular Intensive Care Unit (CVICU),2182-08-15 09:37:33,2182-08-16 17:25:44,1.325127
76538,17577670,24221219,39993265,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2154-01-03 22:50:26,2154-01-06 12:44:43,2.579363


In [18]:
# 분석에 쓸모 없는 열과 예측 시점에서 알 수 없는 열을 삭제
icustays = icustays.drop(['last_careunit', 'outtime', 'intime'], axis=1)

In [19]:
icustays.head()

,subject_id,hadm_id,stay_id,first_careunit,los
0,17867402,24528534,31793211,Trauma SICU (TSICU),1.587454
1,14435996,28960964,31983544,Trauma SICU (TSICU),3.025625
2,17609946,27385897,33183475,Trauma SICU (TSICU),9.741725
3,18966770,23483021,34131444,Trauma SICU (TSICU),0.350544
4,12776735,20817525,34547665,Neuro Stepdown,1.674769


In [20]:
icustays = pd.get_dummies(icustays, columns=['first_careunit'])

In [21]:
# icustays 데이터프레임의 각 변수(열)별 결측치 개수 세기
# icustays는 무결한 데이터임을 확인함

icustays.isnull().sum()

subject_id                                                         0
hadm_id                                                            0
stay_id                                                            0
los                                                                0
first_careunit_Cardiac Vascular Intensive Care Unit (CVICU)        0
first_careunit_Coronary Care Unit (CCU)                            0
first_careunit_Medical Intensive Care Unit (MICU)                  0
first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU)    0
first_careunit_Neuro Intermediate                                  0
first_careunit_Neuro Stepdown                                      0
first_careunit_Neuro Surgical Intensive Care Unit (Neuro SICU)     0
first_careunit_Surgical Intensive Care Unit (SICU)                 0
first_careunit_Trauma SICU (TSICU)                                 0
dtype: int64

In [22]:
icustays.head()

,subject_id,hadm_id,stay_id,los,first_careunit_Cardiac Vascular Intensive Care Unit (CVICU),first_careunit_Coronary Care Unit (CCU),first_careunit_Medical Intensive Care Unit (MICU),first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU),first_careunit_Neuro Intermediate,first_careunit_Neuro Stepdown,first_careunit_Neuro Surgical Intensive Care Unit (Neuro SICU),first_careunit_Surgical Intensive Care Unit (SICU),first_careunit_Trauma SICU (TSICU)
0,17867402,24528534,31793211,1.587454,False,False,False,False,False,False,False,False,True
1,14435996,28960964,31983544,3.025625,False,False,False,False,False,False,False,False,True
2,17609946,27385897,33183475,9.741725,False,False,False,False,False,False,False,False,True
3,18966770,23483021,34131444,0.350544,False,False,False,False,False,False,False,False,True
4,12776735,20817525,34547665,1.674769,False,False,False,False,False,True,False,False,False


> ## diagnoses 전처리

In [23]:
diagnoses['icd_version'].nunique() # 환자 id 수

2

In [24]:
diagnoses['icd_version'].value_counts()[diagnoses['icd_version'].value_counts() > 0]

icd_version
9     3090370
10    2189981
Name: count, dtype: int64

In [25]:
# 외상환자를 추출하기 위한 코드

# ICD-9 codes for trauma are in the range 800-999
# ICD-10 codes for trauma usually start with 'S' or 'T'


def is_trauma_code(icd_code, icd_version):
    if icd_version == 9:
        try:
            code_num = int(icd_code)
            return 800 <= code_num <= 999
        except ValueError:
            return False
    elif icd_version == 10:
        return icd_code.startswith('S') or icd_code.startswith('T')
    else:
        return False

diagnoses = diagnoses[diagnoses.apply(lambda row: is_trauma_code(row['icd_code'], row['icd_version']), axis=1)]

# 외상 코드를 가진 환자 목록만 남게 됨
diagnoses.head()

,subject_id,hadm_id,seq_num,icd_code,icd_version
867,19674536,22975928,5,00845,9
1023,13855272,26012522,3,00845,9
1377,14559749,26693268,9,00845,9
1613,13438122,23638703,4,00869,9
2828,10295020,20194610,7,00845,9


In [26]:
# 분석에 쓸모 없는 열을 삭제
diagnoses = diagnoses.drop(['icd_code', 'icd_version', 'seq_num'], axis=1)

In [27]:
diagnoses.head()

,subject_id,hadm_id
867,19674536,22975928
1023,13855272,26012522
1377,14559749,26693268
1613,13438122,23638703
2828,10295020,20194610


> ## 조인

In [28]:
joined_data = pd.merge(patients, admissions, on='subject_id', how='inner')

joined_data = pd.merge(joined_data, icustays, on=['hadm_id', 'subject_id'], how='inner')

joined_data = pd.merge(joined_data, transfers, on=['hadm_id', 'subject_id'], how='inner')

joined_data = pd.merge(joined_data, diagnoses, on=['hadm_id', 'subject_id'], how='inner')

In [29]:
joined_data

,subject_id,gender,anchor_age,hadm_id,admission_type_AMBULATORY OBSERVATION,admission_type_DIRECT EMER.,admission_type_DIRECT OBSERVATION,admission_type_ELECTIVE,admission_type_EU OBSERVATION,admission_type_EW EMER.,...,first_careunit_Medical Intensive Care Unit (MICU),first_careunit_Medical/Surgical Intensive Care Unit (MICU/SICU),first_careunit_Neuro Intermediate,first_careunit_Neuro Stepdown,first_careunit_Neuro Surgical Intensive Care Unit (Neuro SICU),first_careunit_Surgical Intensive Care Unit (SICU),first_careunit_Trauma SICU (TSICU),eventtype_ED,eventtype_admit,eventtype_transfer
0,10171525,0,27,21263495,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,10171525,0,27,21263495,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
2,10171525,0,27,21263495,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
3,10171525,0,27,21263495,False,False,False,False,False,False,...,False,True,False,False,False,False,False,True,False,False
4,10171525,0,27,21263495,False,False,False,False,False,False,...,False,True,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169391,19999068,1,63,21606769,False,False,False,False,False,True,...,False,True,False,False,False,False,False,False,False,False
169392,19999068,1,63,21606769,False,False,False,False,False,True,...,False,True,False,False,False,False,False,True,False,False
169393,19999068,1,63,21606769,False,False,False,False,False,True,...,False,True,False,False,False,False,False,False,False,True
169394,19999068,1,63,21606769,False,False,False,False,False,True,...,False,True,False,False,False,False,False,False,False,True


# 대규모 데이터 전처리

> ## chartevents 전처리

In [30]:
joined_data.drop(columns=['subject_id', 'hadm_id', 'stay_id'] ).to_csv('joined_data_first_event.csv')

In [31]:
import pandas as pd

# 파일 경로 설정
file_path = 'data/chartevents.csv'

def load_chartevent(file_path):
    # itemid와 대응하는 변수 이름
    itemid_to_variable = {
        220045: 'heart_rate',
        220052: 'arterial_blood_pressure',
        223762: 'temperature_celsius'
    }

    # 각 변수에 대한 빈 DataFrame을 저장할 딕셔너리
    filtered_dfs = {variable: pd.DataFrame() for variable in itemid_to_variable.values()}

    # chunksize 설정
    chunksize = 10 ** 5

    # CSV 파일 읽기
    for chunk in pd.read_csv(file_path, chunksize=chunksize):
        chunk = chunk.drop(['caregiver_id', 'storetime', 'warning'], axis=1, errors='ignore')\
                    .dropna(subset=['value', 'valuenum'])
        for itemid, variable in itemid_to_variable.items():
            # 특정 itemid에 해당하는 행만 필터링
            filtered_chunk = chunk[chunk['itemid'] == itemid]
            # 해당 변수의 DataFrame에 추가
            filtered_dfs[variable] = pd.concat([filtered_dfs[variable], filtered_chunk], ignore_index=True)

    # 각 DataFrame을 별도의 파일로 저장
    for variable, df in filtered_dfs.items():
        df.to_csv(f'{variable}.csv', index=False)

    return filtered_dfs

try:
    heart_rate = pd.read_csv('heart_rate.csv')
    arterial_blood_pressure = pd.read_csv('arterial_blood_pressure.csv')
    temperature_celsius = pd.read_csv('temperature_celsius.csv')
except:
    filtered_dfs = load_chartevent(file_path)
    heart_rate = filtered_dfs['heart_rate']
    arterial_blood_pressure = filtered_dfs['arterial_blood_pressure']
    temperature_celsius = filtered_dfs['temperature_celsius']


In [32]:
heart_rate = heart_rate.drop(columns=['value', 'valueuom', 'itemid', 'subject_id', 'hadm_id'])
arterial_blood_pressure = arterial_blood_pressure.drop(columns=['value', 'valueuom', 'itemid', 'subject_id', 'hadm_id'])
temperature_celsius = temperature_celsius.drop(columns=['value', 'valueuom', 'itemid', 'subject_id', 'hadm_id'])



In [33]:
heart_rate

,stay_id,charttime,valuenum
0,30600691,2165-04-24 05:30:00,65.0
1,30600691,2165-04-24 06:00:00,56.0
2,30600691,2165-04-24 06:09:00,55.0
3,30600691,2165-04-24 07:00:00,57.0
4,30600691,2165-04-24 08:00:00,56.0
...,...,...,...
6798182,30143796,2161-08-30 17:00:00,114.0
6798183,30143796,2161-08-30 18:00:00,96.0
6798184,30143796,2161-08-30 19:00:00,80.0
6798185,30143796,2161-08-30 20:00:00,91.0


In [34]:
arterial_blood_pressure

,stay_id,charttime,valuenum
0,34100191,2196-02-24 16:41:00,82.0
1,34100191,2196-02-24 17:00:00,83.0
2,34100191,2196-02-24 17:16:00,93.0
3,34100191,2196-02-24 17:48:00,88.0
4,34100191,2196-02-24 18:00:00,79.0
...,...,...,...
2387848,38134381,2122-07-28 04:00:00,91.0
2387849,38134381,2122-07-28 05:00:00,83.0
2387850,38134381,2122-07-28 06:00:00,91.0
2387851,38134381,2122-07-28 07:00:00,99.0


In [35]:
temperature_celsius

,stay_id,charttime,valuenum
0,35009126,2186-11-12 20:52:00,35.0
1,35009126,2186-11-12 22:00:00,35.3
2,35009126,2186-11-13 00:00:00,35.3
3,35009126,2186-11-13 02:00:00,35.1
4,35009126,2186-11-13 04:00:00,35.6
...,...,...,...
277616,33357662,2185-11-03 05:00:00,37.5
277617,33357662,2185-11-03 06:00:00,37.6
277618,33357662,2185-11-03 12:00:00,37.0
277619,33357662,2185-11-03 16:00:00,37.6


In [36]:
heart_rate = heart_rate.drop_duplicates()
arterial_blood_pressure = arterial_blood_pressure.drop_duplicates()
temperature_celsius = temperature_celsius.drop_duplicates()

In [37]:
heart_rate = heart_rate.rename(columns={'valuenum': 'heart_rate'})
heart_rate

,stay_id,charttime,heart_rate
0,30600691,2165-04-24 05:30:00,65.0
1,30600691,2165-04-24 06:00:00,56.0
2,30600691,2165-04-24 06:09:00,55.0
3,30600691,2165-04-24 07:00:00,57.0
4,30600691,2165-04-24 08:00:00,56.0
...,...,...,...
6798182,30143796,2161-08-30 17:00:00,114.0
6798183,30143796,2161-08-30 18:00:00,96.0
6798184,30143796,2161-08-30 19:00:00,80.0
6798185,30143796,2161-08-30 20:00:00,91.0


In [38]:
arterial_blood_pressure = arterial_blood_pressure.rename(columns={'valuenum': 'blood_pressure'})
arterial_blood_pressure

,stay_id,charttime,blood_pressure
0,34100191,2196-02-24 16:41:00,82.0
1,34100191,2196-02-24 17:00:00,83.0
2,34100191,2196-02-24 17:16:00,93.0
3,34100191,2196-02-24 17:48:00,88.0
4,34100191,2196-02-24 18:00:00,79.0
...,...,...,...
2387848,38134381,2122-07-28 04:00:00,91.0
2387849,38134381,2122-07-28 05:00:00,83.0
2387850,38134381,2122-07-28 06:00:00,91.0
2387851,38134381,2122-07-28 07:00:00,99.0


In [39]:
temperature_celsius = temperature_celsius.rename(columns={'valuenum': 'temperature_celsius'})
temperature_celsius

,stay_id,charttime,temperature_celsius
0,35009126,2186-11-12 20:52:00,35.0
1,35009126,2186-11-12 22:00:00,35.3
2,35009126,2186-11-13 00:00:00,35.3
3,35009126,2186-11-13 02:00:00,35.1
4,35009126,2186-11-13 04:00:00,35.6
...,...,...,...
277616,33357662,2185-11-03 05:00:00,37.5
277617,33357662,2185-11-03 06:00:00,37.6
277618,33357662,2185-11-03 12:00:00,37.0
277619,33357662,2185-11-03 16:00:00,37.6


In [40]:
jointed_event = pd.merge(temperature_celsius, arterial_blood_pressure, on=['charttime', 'stay_id'], how='inner')

In [41]:
jointed_event = pd.merge(jointed_event, heart_rate, on=['stay_id', 'charttime'], how='inner')

In [42]:
jointed_event.to_csv('data/jointed_event.csv')

In [43]:
joined_data = pd.merge(joined_data, jointed_event,  on=['stay_id'], how='inner')


In [44]:
joined_data

,subject_id,gender,anchor_age,hadm_id,admission_type_AMBULATORY OBSERVATION,admission_type_DIRECT EMER.,admission_type_DIRECT OBSERVATION,admission_type_ELECTIVE,admission_type_EU OBSERVATION,admission_type_EW EMER.,...,first_careunit_Neuro Surgical Intensive Care Unit (Neuro SICU),first_careunit_Surgical Intensive Care Unit (SICU),first_careunit_Trauma SICU (TSICU),eventtype_ED,eventtype_admit,eventtype_transfer,charttime,temperature_celsius,blood_pressure,heart_rate
0,10390732,1,48,26272149,False,False,False,False,False,False,...,False,False,False,False,False,False,2143-07-31 20:45:00,36.2,97.0,95.0
1,10390732,1,48,26272149,False,False,False,False,False,False,...,False,False,False,False,False,False,2143-07-31 21:00:00,36.6,80.0,94.0
2,10390732,1,48,26272149,False,False,False,False,False,False,...,False,False,False,False,False,False,2143-07-31 21:15:00,36.2,107.0,92.0
3,10390732,1,48,26272149,False,False,False,False,False,False,...,False,False,False,False,False,False,2143-07-31 21:30:00,35.2,110.0,94.0
4,10390732,1,48,26272149,False,False,False,False,False,False,...,False,False,False,False,False,False,2143-07-31 21:45:00,35.0,93.0,94.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
664736,19908603,1,67,21951119,False,False,False,False,False,False,...,False,False,False,False,False,True,2151-01-04 05:00:00,37.0,73.0,89.0
664737,19908603,1,67,21951119,False,False,False,False,False,False,...,False,False,False,False,False,True,2151-01-04 06:00:00,36.9,75.0,98.0
664738,19908603,1,67,21951119,False,False,False,False,False,False,...,False,False,False,False,False,True,2151-01-04 07:00:00,36.8,68.0,97.0
664739,19908603,1,67,21951119,False,False,False,False,False,False,...,False,False,False,False,False,True,2151-01-04 09:00:00,36.8,74.0,71.0


In [45]:
len(set(joined_data['stay_id']))

969

In [46]:
joined_data.isna().sum().sum()

0

In [47]:
# 구간별 평균 및 표준편차 계산
def process_data(data, start_hour, duration_hours, interval_count):
    # stay_id별로 그룹화하고, 각 그룹 내에서 필터링 및 통계 계산
    def calculate_stats(group):
        start_time = group['charttime'].min() + pd.Timedelta(hours=start_hour)
        end_time = start_time + pd.Timedelta(hours=duration_hours)

        # 그룹의 최대 charttime이 end_time보다 작은지 여부를 확인하여 컬럼에 저장
        within_time_range = int(group['charttime'].max() < end_time)

        # 각 시간 구간에 대한 변수 평균과 표준 편차 계산
        time_intervals = pd.date_range(start=start_time, end=end_time, periods=interval_count + 1)
        stats = [within_time_range]  # within_time_range 값을 stats 리스트에 추가
        for i in range(interval_count):
            interval_start = time_intervals[i]
            interval_end = time_intervals[i + 1]
            interval_data = group[(group['charttime'] >= interval_start) & (group['charttime'] < interval_end)]

            if interval_data.empty:
                return None

            for col in ['temperature_celsius', 'blood_pressure', 'heart_rate']:
                avg_val = interval_data[col].mean() if not interval_data.empty else np.nan
                std_val = interval_data[col].std() if not interval_data.empty else np.nan
                stats.append(avg_val)
                stats.append(std_val)

        # 평균, 표준편차 및 within_time_range 컬럼 반환
        index_names = ['within_time_range']
        for i in range(interval_count):
            for col in ['temperature_celsius', 'blood_pressure', 'heart_rate']:
                index_names.append(f'{col}_{i}_avg')
                index_names.append(f'{col}_{i}_std')

        return pd.Series(stats, index=index_names)

    # charttime을 datetime으로 변환
    data['charttime'] = pd.to_datetime(data['charttime'])

    # 그룹별 계산 수행
    group_stats = data.groupby('stay_id').apply(calculate_stats)
    # 필터링된 데이터가 없는 그룹 제거
    group_stats = group_stats.dropna(how='all')



    merged_data = pd.merge(data, group_stats, on='stay_id', how='inner')
    merged_data = merged_data.drop_duplicates()
    return merged_data

In [48]:
    
def make_and_save(joined_data, start, end, interval):
    data = process_data(joined_data, start, end, interval)

    data = data.drop(['subject_id', 'hadm_id', 'charttime', \
                           'temperature_celsius', 'blood_pressure', 'heart_rate'], axis=1)
    
    data = data.drop_duplicates()
    data.to_csv(f'data/merged_data_{start}_{end}_{interval}.csv', index=False)
    return data


In [49]:
# 0시부터 12시간 동안의 데이터를 6등분하여 통계량 계산
data_1 = make_and_save(joined_data, 0, 12, 6)

In [50]:
data_1

,gender,anchor_age,admission_type_AMBULATORY OBSERVATION,admission_type_DIRECT EMER.,admission_type_DIRECT OBSERVATION,admission_type_ELECTIVE,admission_type_EU OBSERVATION,admission_type_EW EMER.,admission_type_OBSERVATION ADMIT,admission_type_SURGICAL SAME DAY ADMISSION,...,blood_pressure_4_avg,blood_pressure_4_std,heart_rate_4_avg,heart_rate_4_std,temperature_celsius_5_avg,temperature_celsius_5_std,blood_pressure_5_avg,blood_pressure_5_std,heart_rate_5_avg,heart_rate_5_std
0,1,48,False,False,False,False,False,False,False,False,...,75.666667,2.397317,95.000000,2.197177,35.800000,0.143839,103.666667,40.088981,91.666667,9.109803
172,1,48,False,False,False,False,False,False,False,False,...,75.666667,2.397317,95.000000,2.197177,35.800000,0.143839,103.666667,40.088981,91.666667,9.109803
344,1,48,False,False,False,False,False,False,False,False,...,75.666667,2.397317,95.000000,2.197177,35.800000,0.143839,103.666667,40.088981,91.666667,9.109803
1720,1,43,False,False,False,False,False,True,False,False,...,73.333333,3.120232,101.333333,0.475831,36.550000,0.050709,79.000000,3.042555,103.500000,2.535463
2392,1,43,False,False,False,False,False,True,False,False,...,73.333333,3.120232,101.333333,0.475831,36.550000,0.050709,79.000000,3.042555,103.500000,2.535463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
526648,0,57,False,False,False,False,False,True,False,False,...,79.000000,2.138090,82.500000,1.603567,36.866667,0.049237,80.333333,1.969464,84.000000,1.477098
526709,0,57,False,False,False,False,False,True,False,False,...,79.000000,2.138090,82.500000,1.603567,36.866667,0.049237,80.333333,1.969464,84.000000,1.477098
526770,1,67,False,False,False,False,False,False,True,False,...,65.500000,0.527046,90.000000,2.108185,38.250000,0.052705,73.000000,2.108185,86.000000,1.054093
526817,1,67,False,False,False,False,False,False,True,False,...,65.500000,0.527046,90.000000,2.108185,38.250000,0.052705,73.000000,2.108185,86.000000,1.054093


In [51]:
# 0시부터 6시간 동안의 데이터를 3등분하여 통계량 계산

data_2 = make_and_save(joined_data, 0, 6, 3)

In [52]:
data_2

,gender,anchor_age,admission_type_AMBULATORY OBSERVATION,admission_type_DIRECT EMER.,admission_type_DIRECT OBSERVATION,admission_type_ELECTIVE,admission_type_EU OBSERVATION,admission_type_EW EMER.,admission_type_OBSERVATION ADMIT,admission_type_SURGICAL SAME DAY ADMISSION,...,blood_pressure_1_avg,blood_pressure_1_std,heart_rate_1_avg,heart_rate_1_std,temperature_celsius_2_avg,temperature_celsius_2_std,blood_pressure_2_avg,blood_pressure_2_std,heart_rate_2_avg,heart_rate_2_std
0,1,48,False,False,False,False,False,False,False,False,...,70.2,14.974809,86.2,7.677159,35.75,1.538968e-01,96.0,10.259784,90.0,1.025978
172,1,48,False,False,False,False,False,False,False,False,...,70.2,14.974809,86.2,7.677159,35.75,1.538968e-01,96.0,10.259784,90.0,1.025978
344,1,48,False,False,False,False,False,False,False,False,...,70.2,14.974809,86.2,7.677159,35.75,1.538968e-01,96.0,10.259784,90.0,1.025978
1720,1,43,False,False,False,False,False,True,False,False,...,67.5,4.563833,101.0,0.000000,36.80,7.206219e-15,74.0,3.042555,103.0,1.014185
2392,1,43,False,False,False,False,False,True,False,False,...,67.5,4.563833,101.0,0.000000,36.80,7.206219e-15,74.0,3.042555,103.0,1.014185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600540,0,57,False,False,False,False,False,True,False,False,...,85.0,19.242809,87.0,3.207135,36.60,0.000000e+00,75.0,5.345225,86.0,1.069045
600601,0,57,False,False,False,False,False,True,False,False,...,85.0,19.242809,87.0,3.207135,36.60,0.000000e+00,75.0,5.345225,86.0,1.069045
600662,1,67,False,False,False,False,False,False,True,False,...,65.5,0.527046,96.5,0.527046,37.60,7.489778e-15,73.5,0.527046,94.0,2.108185
600709,1,67,False,False,False,False,False,False,True,False,...,65.5,0.527046,96.5,0.527046,37.60,7.489778e-15,73.5,0.527046,94.0,2.108185
